# One-out-of-two Quantum Oblivious Transfer based on Nonorthogonal States

What is an obivious transfer (OT) protocol?

Most common protocols are all-or-nothing and one-out-of-two. <br>
All-or-nothing OT (1981):<br>
- Alice wants to send secret message $m \in {0,1}$ to Bob
- Bob has a 50% probability of recieving *m*
- Bob either learns *m* with 100% reliability or 0%
- Alice doesn't know if Bob received *m*

One-out-of-two OT (1-2 OT): <br>
- Alice sends $m_0$ and $m_1$ to Bob
- Bob chooses one message and learns nothing about the other
- Alice does not know whichmessage Bob chose

Build 1-2 OT using p-all-or-nothing OT: <br>
- receiver builds two keys to represent his choice, $key_1, key_2$
- Bob learns one key with 100% certainty other with 0%
- Bob asks Alice to encrypt mesages $m_0$ and $m_1$ with using $key_j$
- $j \in {0,1}$
- Bob receive $m_j$

This is a two-level structured protocol

Previous one-out-of two quantum oblivious transfer (QOT) scheme have a two-level structure based in no-go theorem.

The no-go theorem:<br>
Created in 1997 bey Lo.
- says no protocol exists which satisfy:
    - completeness: The protocol should ensure that honest parties can successfully complete the protocol with high probability
    - Bindingness: An unconditionally honest party should be unable to commit to one value and later change it, regardless of any resources they possess
    - Hidingness: An unconditionally honest party should be unable to determine the committed value before the commitment is opened, even with unlimited computational resources
-  highlights the inherent trade-offs and limitations in achieving certain security properties simultaneously

He's proof <br>

TODO

### Proposed Protocol

Step 1: <br>
- Bob creates qubit sequence according to his intentions
    - $j_0$, choose |0> 
    - $j_1$, choose |+> 
- Sequence must be longer than OT sequence
- N is minimum length (Bob receives N messages), M number channel checking qubits, K number loyalty testing qubits
    - $QOT = N \in (|0>,|+>) + M \in (|0>,|1>,|+>,|->) + 2K \in (|0>,|+>) $
- Send sequence to Alice

> Example: <br>
Two qubits: $|0+>_{12} (N)$ to represent his choices and $|0+>_{34} (2K)$ for loyalty testing


Step 2:

- Alice receives sequence: checks for eavesdroppers and tests Bobs loyalty
1. asks Bob to publish bases and states he created, <br>
if error rate higher than threshold, then Eve is present on	channel 
    - communication aborted
2. Checks Bobs loyalty: Alice randomly selects several positions and requests Bob to publish his bases
    - If different results (e.g. not in |0>,|+>), measured is higher than given error rate then Bob is dishonest
    - Communication aborted

> $|0+0+>_{1234}$ , suppose Alice chooses qubit 4, asks Bob to publish base of qubit 4, measures it and compares the published and measured result. <br>
Afterwards qubit 4 is discarded.


Step 3:

- Because of the loyalty test Bob must ask Alice to reorder qubits
- This is what the additional 2K qubits are for, prevent vacancies in the lost of choice intentions
- The sequence after reordering represents Bobs real choices

> Bobs asks Alice to reorder remaining qubits in order 21, <br>
States become $|+0>$, with qubit 3 discarded, <br>
Represent Bob’s choices $j_0$ and $j_1$ respectively


Step 4:

- Alice inputs secret message m0 and m1 through I,X,Y,Z operations according to 00,10,11,01

> Alice performs Z and X so $01_{12}$ and $10_{34} $ <br>
Converts $|+0>_{21}$ to $|-1>_{21}$


Step 5:
- Alice inserts decoy qubits from $|0>,|1>,|+>,|->$ for channel checking 
- Send sequence to Bob


Step 6:

- Bob receives sequence and asks Alice to publish positions and states of decoy qubits
- If error rate higher than channel error rate abort communication
    - Return to step 1
- Otherwise Bob learns contents of the classical message by measuring the qubits with bases he prepared

> Bob performs X and Z basis measurements to learn second and first classical messages, 1 and 1 <br>
($01_{12}$ and $10_{34}$)


Step 7:
- Bob test Alice loyalty
- Bob chooses random positions and asks Alice to publish her operations
- Bob performs operations according to announcements to recover qubits into   $\{|0>,|+>\}$
- Check error rate, if higher than threshold then Alice is dishonest

> Bob asks Alice to publish operations (X) which performs on qubit 1 <br>
Bob performs X on qubit 1 to recover state $|0>$


In [ ]:
#Implementation in Qiskit

from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_textbook.tools import simon_oracle

# Step 1: Bob creates qubit sequence
# Define qubit sequence length
# EXAMPLE:
N = 2  # Bob's qubits
M = 2  # channel checking qubits
K = 2  # loyalty testing qubits

# Create a quantum circuit with N + M + 2K qubits
qc = QuantumCircuit(N + M + 2 * K)

# Bob prepares qubits for choice intentions (|0〉 and |+〉 states)
qc.h(range(N))  # |+〉 state for choice intention j1
# Insert qubits for channel checking and loyalty testing
qc.reset(range(N, N + M + 2 * K))  # Initialize qubits for channel checking and loyalty testing

# Step 2: 

# Step 3: 

# Step 4: 

# Step 5: 

# Step 6: 

# Step 7: 

# Execute the quantum circuit
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(qc).result()
counts = result.get_counts()

# Visualize the measurement outcomes
print(counts)
plot_histogram(counts)


The implementation is dependent on the problem, the implementation of the example:

In [23]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram


# Step 1: Bob creates a qubit sequence
qc_bob = QuantumCircuit(4, 2)  # 4 qubits, 2 classical bits for measurement
qc_bob.h(1)  # Choice intention |+> for j1 (qubit 1)
qc_bob.h(3)  # Loyalty testing |+> (qubit 3)

# Step 2: Alice checks the channel and Bob's loyalty
qc_bob.measure(3, 0)  # Measure qubit 4 (index 3) for loyalty

# Simulate the loyalty check
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc_bob, simulator)
qobj = assemble(compiled_circuit)
result = simulator.run(qobj).result()
counts = result.get_counts()

print("Loyalty check results (qubit 4):", counts)

# Assuming loyalty check passed, we continue
qc_bob.reset(3)  # Reset qubit 4 for reuse

# Step 3: Reorder qubits (keeping original indices for clarity)


# Step 4: Alice performs Z and X operations to encode messages
qc_bob.z(0)  # Applying Z to qubit 0 (to encode message '0')
qc_bob.x(1)  # Applying X to qubit 1 (to encode message '1')

# Step 5: Alice inserts decoy qubits


# Step 6: Bob measures the qubits to learn the classical messages
qc_bob.measure([0, 1], [1, 0])  # Measuring qubits 0 and 1 to classical bits, reordered

# Simulate the protocol after step 6
compiled_circuit = transpile(qc_bob, simulator)
qobj = assemble(compiled_circuit)
result = simulator.run(qobj).result()
counts = result.get_counts()
print("Measurement results after step 6:", counts)

# Step 7: Bob tests Alice's loyalty by performing the announced operations
qc_bob.x(1)  # Bob performs X on qubit 1 to recover state |0>

# Simulate the full protocol after step 7
compiled_circuit = transpile(qc_bob, simulator)
qobj = assemble(compiled_circuit)
result = simulator.run(qobj).result()
counts = result.get_counts()
print("Measurement results after step 7 (qubit 1 only):", {key[-1]: val for key, val in counts.items() if key[-1] == '1'})

# Output the results
plot_histogram(counts).show()

Loyalty check results (qubit 4): {'01': 500, '00': 524}
Measurement results after step 6: {'00': 503, '01': 521}
Measurement results after step 7 (qubit 1 only): {'1': 503}


/var/folders/nx/rglk77t94wngjnsv429fqdhc0000gn/T/ipykernel_83556/2048689240.py:18: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()
/var/folders/nx/rglk77t94wngjnsv429fqdhc0000gn/T/ipykernel_83556/2048689240.py:42: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()
/var/folders/nx/rglk77t94wngjnsv429fqdhc0000gn/T/ipykernel_83556/2048689240.py:52: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be pas

### Security Analysis

#### External Attack

without channel checking or reduced frequency, Eve will be able to illicitly eavesdrop on their messages<br>

Intercept-and-resend attack: <br>
will disturb qubits states <br>
each qubit will be disturbed with a probability of ¼ <br>

reads sequence, measures and resend sequence<br>

Entangling attack: <br>
intercepts transmission, prepares ancillary qubit $|E>$ and perform unitary operation $U_e$

#### Internal Attack

Goal Alice: learn Bob's choices <br>
Goal Bob: learn content of Alice's messages <br>

Alice cheating strategy: <br>
No ability of entanglement:
- perform a single qubit gate {I,X,Y,Z,H}
- success probability: 25% or 29.3%
- attack always detected in protocol (which step?)
    - measurement in the incorrect basis can yield incorrect measurement results that nevertheless help Alice to determine Bob’s initial state

With ability of entanglement:
- perform two or more qubit gates
- Bob required to also have ability of entanglement to resist attacks
- attack will be detected
- success probability 25% or 29.3%
- teleportation attack: (in step 4)
    - instead of inputting secret message into qubit C creates bell state, gives qubit B to Bob to repace C, and keeps qubit A
    - Alice performs bell measurement on C and A
    - According to measurement results gives i,X,Y,Z as her secret message
    - Bob can then recover state $|0>_B or |+>_B$ with apllying said gate
- Bob also has entanglement ability:
    - prepare state ... and send A1 to alice
    - 

continue

Bob cheating strategy:
- prepare entangled qubits
- 





### Efficency Analysis


Comparison of four protocols: Yang's, YYLSZ, YSW and the proposed protocol.

R = message bits that are received

Yang's protocol:
- uses B92 protocol
- four qubits for unambiguous key and only one transmission
- detection strategy with decoy qubits used
- $4 * R + 50$

YYYLSZ protocol:
- four qubits on average to obtain an unambiguous key using
its all-or-nothing QOT strategy i.e. p = 1/4
- two transmissions needed
- strategy is based on the law of large number and will consume a large number of qubits
- $4 * R + 2 * 50$

YSW protocol:
- four qubits for unambiguous key and one transmission
- not include loyalty testing
- Errors may be detected at the application level
- $4 * R + 50$

Proposed protocol:
- based directly on quantum resources and consumes one qubit for
each received bit
- requires two transmissions
- $R + 2 * 50$


<img src="QOTEfficencyComparison.png" width=500px height=130px/>

Quantum Resources for One Message Bit: Number of quantum resources consumed for each received bit, without decoy qubits <br>
Number of Transmissions: Number of transmissions for one sequence <br>
Decoy Qubits: Number of decoy qubits, considering the number of transmissions <br>
Total Cost: The total average quantum resource consumption for R received bits

<img src="QOTComparisonResult.png" width=400px height=300px/>